In [2]:
import qcodes as qc
import qcodes.instrument_drivers.nplab_drivers as npd
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm, DivergingNorm

In [3]:
npd.ppms_init('lockin1', 'lockin2', 'srdc')

Note this uses mT units for getting and setting fields with field() and field_set_stable(). Just divide Oe by 10 to get mT
Connected to: Stanford_Research_Systems SR830 (serial:s/n22100, firmware:ver1.01) in 0.19s
Connected to: Stanford_Research_Systems SR830 (serial:s/n35235, firmware:ver1.07) in 0.05s
Connected to: Stanford_Research_Systems DC205 (serial:s/n20500044, firmware:ver1.80) in 0.02s


In [4]:
srdc.volt.step = 0.05
srdc.volt.inter_delay = 0.1

In [5]:
srdc.volt(1)

In [6]:
srdc.volt(0)

In [5]:
station = qc.Station(ppms, lockin1, lockin2, srdc)

In [8]:
srdc.output(1)

In [13]:
lockin2.amplitude(1)  # set the amplitude to 10 nA for now (100 Mohm resistor, 1V)

# PPMS run 2 of M12 (trying to get T dependence of high-field states)

Previous fridge contacts translated to (->) ppms contacts
12->7, 48->6, 9->9, 22->4, 20->3, 19->14, 13->13, 12->12

back gate: 8

Source/drain to 7/14

Both Rxx  
lockin1 connected to 4-3  
lockin2 connected to 12-13

Using 100 MOhm limiting resistor for the AC excitation
Using 1 MOhmr resistor for the gate

In [15]:
# set the temperature sweep to 2K and then do an initial gate sweep
# use lockin time constant 100 ms, 12 dB slope so we can do a faster sweep here
# nothing came out of this sweep...
srdc.volt(-5)
time.sleep(8)
gvals = np.linspace(-5, 5, 101)
data1, plot1 = npd.single_param_sweep(srdc.volt, gvals, 0.75, lockin1.X, lockin1.Y, lockin2.X, lockin2.Y, DataName='M12_hightempgsweeprough_lockin1_4-3_lockin2_12-13_10nA')

Started at 2020-02-24 10:16:31
DataSet:
   location = 'data/2020-02-24/#002_M12_hightempgsweeprough_lockin1_4-3_lockin2_12-13_10nA_10-16-29'
   <Type>   | <array_id>    | <array.name> | <array.shape>
   Setpoint | srdc_volt_set | volt         | (101,)
   Measured | lockin1_X     | X            | (101,)
   Measured | lockin1_Y     | Y            | (101,)
   Measured | lockin2_X     | X            | (101,)
   Measured | lockin2_Y     | Y            | (101,)
Finished at 2020-02-24 10:18:11


In [17]:
srdc.volt(0)

In [18]:
srdc.volt(5)

In [19]:
srdc.volt(4.5)

In [20]:
srdc.volt(4)

In [21]:
srdc.volt(0)

Now at 2K

In [22]:
srdc.volt(-5)

In [23]:
lockin2.amplitude(0.2)

In [30]:
srdc.volt(-12)

In [28]:
lockin2.amplitude(1)

In [31]:
gvals = np.linspace(-12, 12, 241)
data2, plot2 = npd.single_param_sweep(srdc.volt, gvals, 1, lockin2.X, lockin2.Y, DataName='M12_2K_lockin2_4-3_10nA')

Started at 2020-02-24 11:57:20
DataSet:
   location = 'data/2020-02-24/#004_M12_2K_lockin2_4-3_10nA_11-57-19'
   <Type>   | <array_id>    | <array.name> | <array.shape>
   Setpoint | srdc_volt_set | volt         | (241,)
   Measured | lockin2_X     | X            | (241,)
   Measured | lockin2_Y     | Y            | (241,)
Finished at 2020-02-24 11:59:50
Keyboard Interrupt


In [32]:
srdc.volt(0)

In [33]:
ppms.temperature(10)

In [34]:
srdc.volt(-12)
time.sleep(15)
gvals = np.linspace(-12, 12, 241)
data3, plot3 = npd.single_param_sweep(srdc.volt, gvals, 1, lockin2.X, lockin2.Y, DataName='M12_10K_lockin2_4-3_10nA')

Started at 2020-02-24 12:02:33
DataSet:
   location = 'data/2020-02-24/#005_M12_10K_lockin2_4-3_10nA_12-02-31'
   <Type>   | <array_id>    | <array.name> | <array.shape>
   Setpoint | srdc_volt_set | volt         | (241,)
   Measured | lockin2_X     | X            | (241,)
   Measured | lockin2_Y     | Y            | (241,)
Finished at 2020-02-24 12:05:54
Keyboard Interrupt


In [35]:
srdc.volt(0)

In [36]:
lockin2.amplitude(0.1)

There may be a short from the gate to the device... Taking it out

In [35]:
ppms.close()
lockin1.close()
lockin2.close()
srdc.close()

#  After taking it out and reloading it

After taking it out, looking in the microscope, and reloading it, the gate doesn't seem shorted (though who knows for sure until we try it a little more). Now contact 7 is not working, but the others are. Use 14 as the source (through 100 MOhm) and 9 as the drain.

In [12]:
lockin2.amplitude(1)  # using 10 nA

In [19]:
srdc.volt(-5)

In [20]:
# nothing happens with the gate... maybe the gate bond is disconnected or something weird like that...

In [22]:
srdc.volt(0)

In [28]:
srdc.volt(3)

In [29]:
srdc.volt(0)

After rebonding contact 7 and 8, contact 7 works. However, the gate is noticably shorted to the other electrodes, as measured by a 2-point measurement. And the gate doesn't seem to affect the other contacts

In [34]:
srdc.volt(0)